In [ ]:
!pip install confluent_kafka

In [1]:
from confluent_kafka import Producer

# Configuración: se debe reemplazar con los valores de cada cuenta
BROKERS  = "sulky-01.srvs.cloudkafka.com:9094,sulky-02.srvs.cloudkafka.com:9094,sulky-03.srvs.cloudkafka.com:9094"
USERNAME = "hqeayvbp"
PASSWORD = "ljw2vm2Kg78lz1MmxfflvKOtYpArayjh"
TOPIC    = "hqeayvbp-click"

In [2]:
# Tópico que será utilizado
topic = TOPIC.split(",")[0]

# Configuración de la comunicación con Kafka
conf = {'bootstrap.servers': BROKERS,
        'session.timeout.ms': 6000,
        'default.topic.config': {'auto.offset.reset': 'smallest'},
        'security.protocol': 'SASL_SSL',
        'sasl.mechanisms': 'SCRAM-SHA-256',
        'sasl.username': USERNAME,
        'sasl.password': PASSWORD
        }

# Creación de un productor (publica en el tópico)
prod = Producer(**conf)

In [3]:
# Función que se ejecuta cada vez que se publica en el tópico
def delivery_callback(err, msg):
  if err:
    print(f'%% No se pudo entregar el mensaje: {err}')
  else:
    print(f'%% Mensaje entregado a {msg.topic()} {msg.partition()}')

In [ ]:
try:
  # Mensaje a publicar
  mensaje = "Nuevo mensaje publicado en el topico"
  # Publicación del mensaje
  prod.produce(topic, mensaje.rstrip(), callback=delivery_callback)

except BufferError as e:
  print(f'%% La cola del productor está llena ({len(prod)} mesnajes a la espera): re intentar')
prod.poll(0)

# Mensaje final
print(f'%% Esperando {len(prod)} entregas')
# "Borra" el log (buffer)
prod.flush()

In [ ]:
import time

# Envío seguido de mensajes cada 0.5 segundos
N = 20
for i in range(N):
  mensaje = "iteracion: " + str(i+1)
  prod.produce(topic, mensaje, callback=delivery_callback)
  time.sleep(0.5)

prod.flush()